# Assignment description

In this assignment, you will be wrting SQL for a real-world problem using the full iowa dataset.   In contrast to previous assignments, where we directly wrote SQL queries using the magic `%%sql` cells, we will be directly connecting to the database and running queries through the Python database client.   This is how the magic `%%sql` cells are implemented under the covers anyways.

We will also be using [DuckDB](https://duckdb.org/), a new database system designed for analytics.  It is very similar to the SQLite database we have used in the past, however it is _must faster_ when analyzing the entire dataset.

# Setup code 
The following three blocks only need to run once.   It will install duckdb, download the data files, and load it into duckdb.



In [2]:
!pip install duckdb
!rm iowa.csv* iowa.duckdb*
!wget https://www.dropbox.com/s/0f4g8xa5m2s898i/iowa.csv

  Using cached duckdb-1.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (762 bytes)
Using cached duckdb-1.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.5 MB)
rm: cannot remove 'iowa.csv*': No such file or directory
rm: cannot remove 'iowa.duckdb*': No such file or directory
--2024-09-06 21:22:13--  https://www.dropbox.com/s/0f4g8xa5m2s898i/iowa.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/s2pq83g2orsm7fodpb2k4/iowa.csv?rlkey=uvjpwofnuqwpxg0by5zc93zfv [following]
--2024-09-06 21:22:13--  https://www.dropbox.com/scl/fi/s2pq83g2orsm7fodpb2k4/iowa.csv?rlkey=uvjpwofnuqwpxg0by5zc93zfv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc05319932fc5794864bc3804e51.dl

In [3]:
import duckdb
db = duckdb.connect('iowa.db')
db.execute("CREATE TABLE iowa AS SELECT * FROM read_csv_auto('iowa.csv')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
def runq(q):
  cursor = db.execute(q)
  df = cursor.fetchdf()
  print(df)
  return df

### **Q1**: For each zipcode, compute the two most purchased category_name by total `sale_bottles`, for two category names with the same total `sale_bottles`, return the category name with larger lexicographical. Your result shall include two columns: `zipcode` and `category_name`, sort the final results by `zipcode` in ascending order and only display 10 records.

You may either write a single nested query, or follow the step by step decomposition we provide. You will receive full credits as long as the query is correct

a. We first need to get have a table with aggregated sale_bottles groupped by zipcode & category_name

In [38]:
q1a = """CREATE OR REPLACE VIEW zip_cate_sum AS 
    SELECT zipcode, category_name, sum(sale_bottles) sum_qty
    FROM iowa
    GROUP BY zipcode, category_name
"""
runq(q1a)

Empty DataFrame
Columns: [Count]
Index: []


,Count


b. With `zip_cate_sum` table, we can get the max `sum_qty` in a given zipcode with other groupby.

In [39]:
q1b = """CREATE OR REPLACE VIEW zip_cate_sum_max AS (
    SELECT zcs.zipcode, max(zcs.sum_qty) max_qty
    FROM zip_cate_sum zcs
    GROUP BY zcs.zipcode
)"""
runq(q1b)

Empty DataFrame
Columns: [Count]
Index: []


,Count


c. use max bottle_qty sum and zipcode information, we can join above two tables to get `category_name` of max total `sale_bottles`.

In [48]:
q1c = """CREATE OR REPLACE VIEW max_cate_zip AS (
    SELECT c.zipcode, max(c.category_name) category_name
    FROM zip_cate_sum c, zip_cate_sum_max b
    WHERE b.zipcode = c.zipcode AND b.max_qty = c.sum_qty
    GROUP BY c.zipcode
)"""
runq(q1c)

Empty DataFrame
Columns: [Count]
Index: []


,Count


d. create a table that removes the `category_name` of max `sale_bottles` from the aggregation of `sale_bottles` group by zipcode

In [41]:
q1d = """
    CREATE OR REPLACE VIEW zip_cate_sum_sec_max AS (
        SELECT zipcode, max(sum_qty) max_qty
        FROM (
            SELECT zcs.zipcode, zcs.sum_qty
            FROM zip_cate_sum zcs, max_cate_zip mcz
            WHERE zcs.zipcode = mcz.zipcode
            AND zcs.category_name != mcz.category_name
        )
        GROUP BY zipcode
    )
"""
runq(q1d)

Empty DataFrame
Columns: [Count]
Index: []


,Count


e. use max bottle_qty sum and zipcode information, we can join above two tables to get `category_name` of second max total `sale_bottles`.

In [51]:
q1e = """
    CREATE OR REPLACE VIEW sec_max_cate_zip AS (
        SELECT c.zipcode, max(c.category_name) category_name
        FROM zip_cate_sum c, zip_cate_sum_sec_max b
        WHERE b.zipcode = c.zipcode AND b.max_qty = c.sum_qty
        GROUP BY c.zipcode
    )
"""
runq(q1e)

Empty DataFrame
Columns: [Count]
Index: []


,Count


f. Union the zipcode for max category name and second max category name

In [56]:
q1f = """
SELECT * FROM
max_cate_zip 
UNION 
SELECT * FROM
sec_max_cate_zip
ORDER BY zipcode
limit 10
"""
runq(q1f)

  zipcode      category_name
0   50002     VODKA 80 PROOF
1   50002  CANADIAN WHISKIES
2   50003  CANADIAN WHISKIES
3   50003     VODKA 80 PROOF
4   50006     VODKA 80 PROOF
5   50006  CANADIAN WHISKIES
6   50009     VODKA 80 PROOF
7   50009  CANADIAN WHISKIES
8   50010  CANADIAN WHISKIES
9   50010     VODKA 80 PROOF


,zipcode,category_name
0,50002,VODKA 80 PROOF
1,50002,CANADIAN WHISKIES
2,50003,CANADIAN WHISKIES
3,50003,VODKA 80 PROOF
4,50006,VODKA 80 PROOF
5,50006,CANADIAN WHISKIES
6,50009,VODKA 80 PROOF
7,50009,CANADIAN WHISKIES
8,50010,CANADIAN WHISKIES
9,50010,VODKA 80 PROOF
